In [1]:
import simi

import ast
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from sentence_transformers import evaluation, losses, models, InputExample, SentenceTransformer, util as stu
from sklearn.metrics import classification_report, f1_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
import torch
from tqdm.auto import tqdm, trange
tqdm.pandas()

pd.set_option('display.max_columns', 30)

RANDOM_STATE = 1

random.seed(RANDOM_STATE)

2023-12-17 22:45:09.190610: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [12]:
def _msc_lvl(level):
    if   level==1: return 2
    elif level==2: return 3
    else:          return 5


def predict_msc(df, col, n_results, top_k, level=1):
    e = _msc_lvl(level)
    if top_k == 1:
        prediction = df[col].map(lambda c: Counter([m[0:e] for m in c[:n_results]]).most_common(top_k)[0][0])
    else:
        prediction = df[col].map(lambda c: [t[0] for t in Counter([m[0:e] for m in c[:n_results]]).most_common(top_k)])
    return prediction

def model_predict_msc(eval_models, df, n_results, top_k, level=1, msc_col="MSC"):
    e = _msc_lvl(level)
    res = pd.DataFrame({"actual": df[msc_col].map(lambda c: c[0:e])})
    for n in eval_models.keys():
        res["prediction ({})".format(n)] = predict_msc(df, "MSC ({})".format(n), n_results, top_k, level=level)
        if top_k > 1:
            res["prediction ({})".format(n)] = pd.Series(map(lambda t: t[0] if t[0] in t[1] else t[1][0], zip(res["actual"].values, res["prediction ({})".format(n)].values)))
    res.index = df.index
    return res

def model_predict_msc_separate_params(eval_models, df, model_params, top_k, level=1, msc_col="MSC"):
    e = _msc_lvl(level)
    res = pd.DataFrame({"actual": df[msc_col].map(lambda c: c[0:e])})
    for n in eval_models.keys():
        res["prediction ({})".format(n)] = predict_msc(df, "MSC ({})".format(n), model_params[n][0], top_k, level=level)
        if top_k > 1:
            res["prediction ({})".format(n)] = pd.Series(map(lambda t: t[0] if t[0] in t[1] else t[1][0], zip(res["actual"].values, res["prediction ({})".format(n)].values)))
    res.index = df.index
    return res

def model_f1(eval_models, df, n_results, top_k, level=1, msc_col="MSC"):
    ap = model_predict_msc(eval_models, df, n_results, top_k, level=level, msc_col=msc_col)
    res = pd.DataFrame({"f1 ({})".format(n): [f1_score(ap["actual"], ap["prediction ({})".format(n)], average="weighted")] for n in eval_models.keys()})
    return res

def model_correct_prediction_separate_params(eval_models, df, model_params, top_k, level=1, msc_col="MSC"):
    ap = model_predict_msc_separate_params(eval_models, df, model_params, top_k, level=level, msc_col=msc_col)
    if top_k == 1:
        cmp_fn = lambda x,y: x==y
    else:
        cmp_fn = lambda x,y: x in y
        
    res = pd.DataFrame({correct prediction {}}.format(n): map(cmp_fn, zip(ap["actual"], ap["prediction ({})".format(n)])))
    return res

def model_f1_separate_params(eval_models, df, model_params, top_k, level=1, msc_col="MSC"):
    ap = model_predict_msc_separate_params(eval_models, df, model_params, top_k, level=level, msc_col=msc_col)
    res = pd.DataFrame({"f1 ({})".format(n): [f1_score(ap["actual"], ap["prediction ({})".format(n)], average="weighted")] for n in eval_models.keys()})
    return res

def model_f1_param(eval_models, df, n_range, top_k, level=1, msc_col="MSC"):
    res = pd.concat([model_f1(eval_models, df, i, top_k, level=level, msc_col="MSC") for i in tqdm(n_range, leave=False)], ignore_index=True)
    res.index = n_range
    return res

def best_model_f1_param(eval_models, df, n_range, top_k, level=1, msc_col="MSC"):
    params = model_f1_param(eval_models, df, n_range, top_k, level=level, msc_col=msc_col)
    best = {}
    for n in eval_models.keys():
        cn = "f1 ({})".format(n)
        print(n, list(params[cn].sort_values(ascending=False).items())[:5])
        best[n] = (params[cn].idxmax(), params[cn].max())
    return best

In [3]:
# Prerequisites for the models
bert_base = models.Transformer('bert-base-uncased')
bert_mp = SentenceTransformer(modules=[bert_base, models.Pooling(bert_base.get_word_embedding_dimension())])

mathbert_base = models.Transformer('witiko/mathberta')
mathbert_mp = SentenceTransformer(modules=[mathbert_base, models.Pooling(mathbert_base.get_word_embedding_dimension())])
mathbert_cls = SentenceTransformer(modules=[mathbert_base, models.Pooling(mathbert_base.get_word_embedding_dimension(), pooling_mode="cls")])

bert_mlm_base = models.Transformer("./bert+re-train_mlm_abstracts_arxiv")
bert_mlm_mp = SentenceTransformer(modules=[bert_mlm_base, models.Pooling(bert_mlm_base.get_word_embedding_dimension())])

sbert = SentenceTransformer('all-mpnet-base-v2')
# alternative: sbert = SentenceTransformer('all-distilroberta-v1')

specter2_base = models.Transformer('allenai/specter2_base')

# Definition of models that are evaluated
# combine models into dict
eval_models = {
    "Bert+MP": bert_mp,
    "Bert+MP+class-arx": SentenceTransformer('./bert+mean-pooling+retrain_class_arxiv'),
    "Bert+MP+class-zbm": SentenceTransformer('./bert+mean-pooling+retrain_class_zbmath'),
    
    "Mathbert+CLS": mathbert_cls,
    "Mathbert+MP+class-arx": SentenceTransformer('./mathbert+mean-pooling+retrain_class_arxiv'),
    "Mathbert+MP+class-zbm": SentenceTransformer('./mathbert+mean-pooling+retrain_class_zbmath'),
    
    "Bert+TSDAE+MP": SentenceTransformer('./bert+mean-pooling+re-train_tsdae_abstracts_arxiv'),
    "Bert+TSDAE+MP+class-arx": SentenceTransformer('./bert+mean-pooling+re-train_tsdae_abstracts_arxiv+retrain_class_arxiv'),
    "Bert+TSDAE+MP+class-zbm": SentenceTransformer('./bert+mean-pooling+re-train_tsdae_abstracts_arxiv+retrain_class_zbmath'),
    
    "Bert+MLM+MP": bert_mlm_mp,
    "Bert+MLM+MP+class-arx": SentenceTransformer('./bert+re-train_mlm_abstracts_arxiv+mean-pooling+retrain_class_arxiv'),
    "Bert+MLM+MP+class-zbm": SentenceTransformer('./bert+re-train_mlm_abstracts_arxiv+mean-pooling+retrain_class_zbmath'),
    "Bert+MLM+MP+class-zbm+anch-arx": SentenceTransformer('./bert+re-train_mlm_abstracts_arxiv+mean-pooling+retrain_class_zbmath_anchor_arxiv'),
    "Bert+MLM+MP+class-arx+anch-arx+class-zbm": SentenceTransformer('./bert+re-train_mlm_abstracts_arxiv+mean-pooling+retrain_class_arxiv_anchor_arxiv_class_zbmath'),
    
    "SBert": sbert,
    "SBert+ret_class-arx": SentenceTransformer('./sbert+retrain_class_arxiv'),
    "SBert+ret_class-zbm": SentenceTransformer('./sbert+retrain_class_zbmath'),
    
    "Specter 2+MP": SentenceTransformer(modules=[specter2_base, models.Pooling(specter2_base.get_word_embedding_dimension())]),
    "Specter 2.0+MP+class-arx": SentenceTransformer('./specter2+mp+retrain_class_arxiv'),
    "Specter 2.0+MP+class-zbm": SentenceTransformer('./specter2+mp+retrain_class_zbmath'),
    "Specter 2+MP+class-zbm+anch-arx": SentenceTransformer('./specter2+mp+retrain_class_zbmath_anchor_arxiv'),
    "Specter 2+MP+anch-arx+class-arx+class-zbm": SentenceTransformer('./specter2+mp+retrain_anchor_arxiv_class_arxiv_class_zbmath'),
    
    "SGPT": SentenceTransformer('Muennighoff/SGPT-125M-weightedmean-nli-bitfit'),
    "SGPT+class-arx": SentenceTransformer('./gpt+retrain_class_arxiv'),
    "SGPT+class-zbm": SentenceTransformer('./gpt+retrain_class_zbmath'),
}

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at witiko/mathberta were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_h

In [4]:
%%time
X_test = pd.read_pickle("zbmath-200-recommendations-2000-2019-multi-test.pkl")
X_eval = pd.read_pickle("zbmath-200-recommendations-2000-2019-multi-eval.pkl")

print(len(X_test), len(X_eval))

X_test["MSC2"] = X_test["MSC2"].map(lambda c: ast.literal_eval(c))
X_eval["MSC2"] = X_eval["MSC2"].map(lambda c: ast.literal_eval(c))

102609 102609
CPU times: user 59 s, sys: 11.6 s, total: 1min 10s
Wall time: 1min 10s


In [5]:
X_test.sample(5)

,title,MSC,MSC2,embedding (Bert+MP),embedding (Bert+MP+class-arx),embedding (Bert+MP+class-zbm),embedding (Mathbert+CLS),embedding (Mathbert+MP+class-arx),embedding (Mathbert+MP+class-zbm),embedding (Bert+TSDAE+MP),embedding (Bert+TSDAE+MP+class-arx),embedding (Bert+TSDAE+MP+class-zbm),embedding (Bert+MLM+MP),embedding (Bert+MLM+MP+class-arx),embedding (Bert+MLM+MP+class-zbm),...,MSC (Bert+MLM+MP+class-arx),MSC (Bert+MLM+MP+class-zbm),MSC (Bert+MLM+MP+class-zbm+anch-arx),MSC (Bert+MLM+MP+class-arx+anch-arx+class-zbm),MSC (SBert),MSC (SBert+ret_class-arx),MSC (SBert+ret_class-zbm),MSC (Specter 2+MP),MSC (Specter 2.0+MP+class-arx),MSC (Specter 2.0+MP+class-zbm),MSC (Specter 2+MP+class-zbm+anch-arx),MSC (Specter 2+MP+anch-arx+class-arx+class-zbm),MSC (SGPT),MSC (SGPT+class-arx),MSC (SGPT+class-zbm)
61258,Last passage percolation in macroscopically in...,60K35,"(82D60, 60F10, 82B43)","[-0.36934817, -0.088138424, 0.14222482, -0.212...","[0.012780122, -0.18500479, -0.6725353, -0.4644...","[-0.5084884, 0.2700277, 0.44205242, 0.42792243...","[-0.022734499, 0.021252265, -0.021943629, 0.01...","[0.12570016, 0.10171499, 0.907296, 1.1970586, ...","[0.40993747, -0.5918986, -0.044760793, 0.76490...","[-0.20297414, 0.059957117, 0.040534817, -0.040...","[0.3471524, -0.47479653, -0.10293299, -0.04506...","[-0.09127718, 0.24273452, -0.0815342, 0.591843...","[-0.21035297, 0.08755582, -0.20333107, 0.33957...","[0.49080715, 0.72965634, -0.055031188, 0.41614...","[-0.22494234, -0.27616727, -0.23733015, 0.0595...",...,"[60K35, 60K35, 82B43, 82B43, 82C70, 82B43, 82B...","[60K35, 60K35, 60K35, 82B43, 60K35, 60K35, 60K...","[60G50, 60K35, 60K35, 60K35, 60K35, 60C05, 82B...","[60K35, 60K35, 60K35, 60K35, 60K35, 82B43, 60K...","[60K35, 60K35, 82B43, 60K35, 60K35, 82B43, 82C...","[60K35, 60K35, 82B43, 60K35, 60K35, 76F25, 60K...","[82C43, 82C43, 60K35, 60K35, 60K35, 60K35, 60K...","[60K35, 82B43, 60K35, 82B43, 60K35, 60K35, 60K...","[60K35, 60K35, 60K35, 60F10, 60F10, 05C80, 60F...","[82B43, 60K35, 82B43, 60K35, 82B43, 82B43, 82B...","[82B43, 60K35, 82B43, 60K35, 82B43, 60K35, 82C...","[82B43, 60K35, 82C27, 60K35, 60K35, 60K35, 82B...","[82B43, 35B25, 82C43, 76R50, 60H15, 82B43, 74S...","[82C43, 82B43, 60K35, 60K35, 60K35, 82C20, 60K...","[60K35, 82B21, 60K35, 60K35, 60K35, 60K35, 60K..."
102031,Two-scale large deviations for chemical reacti...,81V70,"(81T70, 81S40, 82C31, 92D20, 80A30, 35B25, 60F10)","[-0.124032296, -0.33491635, -0.21684809, -0.17...","[-0.48845252, -0.45746002, 0.59039325, -0.1627...","[0.24347334, 0.31035084, -0.3905167, -0.891570...","[-0.017628863, 0.0207977, -0.016881885, 0.0203...","[0.23377636, 0.1744643, 0.94811076, 1.2998352,...","[0.6495893, 0.025765348, 0.07145103, 1.1293417...","[0.24030764, 0.104242586, -0.583749, 0.0183610...","[0.25427192, -0.55246294, -0.101694144, -0.003...","[-0.12151952, 0.5177228, -0.4954238, 0.0178991...","[-0.103333294, 0.025324713, -0.13070707, 0.193...","[0.4699212, 0.5291902, 0.10797089, 0.38027844,...","[0.5363046, -0.23643987, -0.50654715, -0.47273...",...,"[76M35, 82B26, 60F10, 60H10, 60K25, 62G20, 82C...","[82C22, 37A50, 60F10, 82B10, 82B05, 60K35, 82C...","[82C22, 82C05, 60F10, 82C20, 60F10, 82C22, 65C...","[60F10, 60F10, 60F10, 60H15, 82C20, 60K35, 37M...","[80A25, 60F10, 81S40, 60K35, 82C43, 81V55, 82C...","[60H35, 62M05, 60F10, 60F10, 60H07, 60G50, 60J...","[60F10, 35R60, 60G17, 76M28, 60F10, 60K35, 60F...","[92E20, 82C31, 35K57, 82C31, 81V70, 80A30, 35B...","[60H10, 65C30, 60G15, 92D30, 60G35, 60G50, 60K...","[82C10, 60F10, 92E20, 82C41, 60F10, 35Q92, 92E...","[93E03, 82C20, 60F10, 60F10, 60F10, 60F10, 37C...","[60F10, 60G60, 60F10, 93E03, 60F10, 60J10, 60F...","[65C05, 76Y05, 65M06, 35B33, 35R30, 74J10, 82C...","[92C45, 92D30, 35K57, 34K10, 35K51, 65L50, 35K...","[80A10, 80A30, 80A25, 65P20, 80A32, 65L05, 92C..."
7378,Selective sequential pseudocompactness,54D30,"(20A05, 20C05, 54A20, 54H11)","[-0.028313607, -0.26613683, -0.13911855, 0.193...","[0.13446139, 0.38506082, -0.31634188, 0.19

## TOP-1

### Parameter search

In [6]:
%%time
best_1_1 = best_model_f1_param(eval_models, X_test, list(range(5, 51)), 1, level=1)

  0%|          | 0/46 [00:00<?, ?it/s]

Bert+MP [(8, 0.5747805310625493), (7, 0.5747412441007752), (9, 0.5745677753270664), (10, 0.5743018415916484), (11, 0.5733222939776168)]
Bert+MP+class-arx [(8, 0.5119573121274651), (10, 0.5116750408441322), (9, 0.5116050660905571), (11, 0.5109598812056485), (12, 0.5100384733936214)]
Bert+MP+class-zbm [(10, 0.6133098253028466), (8, 0.6132669191702741), (7, 0.6131049192076394), (9, 0.613038175384465), (11, 0.6127221253703838)]
Mathbert+CLS [(10, 0.6207138804968556), (13, 0.6202829034477036), (11, 0.6200821076049354), (12, 0.6199801299666091), (14, 0.6199600820250702)]
Mathbert+MP+class-arx [(10, 0.49168459031977446), (13, 0.4912825293350538), (14, 0.49124870885777866), (12, 0.49124509605095856), (11, 0.4909608627069158)]
Mathbert+MP+class-zbm [(11, 0.5936692555449425), (12, 0.5934576555668802), (10, 0.5932694732227923), (15, 0.5929385073030828), (9, 0.5928594600634873)]
Bert+TSDAE+MP [(11, 0.6556031850962624), (12, 0.6552915040894214), (10, 0.6552172940702502), (9, 0.6551306720667588), (1

In [7]:
%%time
best_1_2 = best_model_f1_param(eval_models, X_test, list(range(5, 51)), 1, level=2)

  0%|          | 0/46 [00:00<?, ?it/s]

Bert+MP [(9, 0.3858302531941316), (10, 0.3857767624390032), (11, 0.3851526155061588), (8, 0.385077289314037), (12, 0.3844509327504135)]
Bert+MP+class-arx [(7, 0.30176834806662295), (6, 0.30143490048949895), (8, 0.30133841623607477), (10, 0.3012989314451711), (9, 0.301016560027949)]
Bert+MP+class-zbm [(9, 0.4467189936740102), (10, 0.44665676531864634), (11, 0.44657537985822826), (8, 0.4462931272130655), (12, 0.44579777178718066)]
Mathbert+CLS [(11, 0.4229803023449042), (13, 0.42261066504424355), (12, 0.42250561112474827), (10, 0.42229512645732725), (14, 0.42223485537212246)]
Mathbert+MP+class-arx [(11, 0.2707591163532126), (10, 0.2706082590510364), (12, 0.27059516516173926), (8, 0.27033961672684226), (13, 0.27028942224839436)]
Mathbert+MP+class-zbm [(18, 0.4261051290146247), (17, 0.42566706347981814), (14, 0.4255254167553598), (12, 0.4253094024295022), (19, 0.4252839954476475)]
Bert+TSDAE+MP [(15, 0.4681575773069091), (14, 0.46788339596660605), (12, 0.4678643339115633), (10, 0.467778614

In [8]:
%%time
best_1_3 = best_model_f1_param(eval_models, X_test, list(range(5, 51)), 1, level=3)

  0%|          | 0/46 [00:00<?, ?it/s]

Bert+MP [(9, 0.22273314785194104), (7, 0.222451894168282), (11, 0.22241247049586962), (10, 0.2222696835980275), (8, 0.22222598602195365)]
Bert+MP+class-arx [(7, 0.155341238865141), (6, 0.15484719502388214), (8, 0.1543151435005947), (5, 0.15384159985065818), (9, 0.1536608075219553)]
Bert+MP+class-zbm [(9, 0.2772804634973854), (10, 0.27684081032629804), (8, 0.27629931172768746), (7, 0.27565390182347876), (11, 0.2756537313222978)]
Mathbert+CLS [(11, 0.2508073524963204), (10, 0.2506292959801685), (12, 0.2505847623418292), (14, 0.25026297553792737), (9, 0.2500672973489478)]
Mathbert+MP+class-arx [(7, 0.12468846683845947), (11, 0.12468037605530297), (10, 0.12452147564451493), (8, 0.12450228808165), (9, 0.1244658333262846)]
Mathbert+MP+class-zbm [(14, 0.2533571567399322), (12, 0.253043178678029), (11, 0.2528907298004954), (15, 0.2528143970280776), (13, 0.25277793312756397)]
Bert+TSDAE+MP [(16, 0.29374673548818236), (15, 0.2934334942485396), (11, 0.29334880149261905), (17, 0.29317382912789897)

### X_test

In [13]:
model_f1_separate_params(eval_models, X_test, best_1_1, 1, level=1, msc_col="MSC")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.574781,0.511957,0.61331,0.620714,0.491685,0.593669,0.655603,0.475965,0.616543,0.665049,0.555552,0.642626,0.629962,0.644116,0.697461,0.552583,0.634195,0.682664,0.483694,0.618219,0.642673,0.644011,0.606158,0.537841,0.626631


In [14]:
model_f1_separate_params(eval_models, X_test, best_1_2, 1, level=2, msc_col="MSC")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.38583,0.301768,0.446719,0.42298,0.270759,0.426105,0.468158,0.263817,0.448495,0.468385,0.334058,0.469159,0.456242,0.467009,0.51791,0.347939,0.46625,0.494295,0.260349,0.446971,0.469588,0.468274,0.426218,0.336916,0.454631


In [15]:
model_f1_separate_params(eval_models, X_test, best_1_3, 1, level=3, msc_col="MSC")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.222733,0.155341,0.27728,0.250807,0.124688,0.253357,0.293747,0.126022,0.27692,0.283801,0.163734,0.28729,0.27826,0.286702,0.338924,0.190584,0.293968,0.316086,0.117502,0.270075,0.292097,0.289439,0.263871,0.186002,0.283651


### X_eval

In [16]:
model_f1_separate_params(eval_models, X_eval, best_1_1, 1, level=1, msc_col="MSC")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.572244,0.509651,0.615331,0.621862,0.493201,0.591002,0.653006,0.479597,0.617393,0.66374,0.55669,0.641348,0.632068,0.642171,0.697088,0.554932,0.634412,0.681159,0.481872,0.617571,0.641212,0.643541,0.603539,0.534302,0.624062


In [17]:
model_f1_separate_params(eval_models, X_eval, best_1_2, 1, level=2, msc_col="MSC")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.380825,0.301299,0.443907,0.422182,0.272059,0.421979,0.464677,0.264571,0.444442,0.465917,0.334914,0.468474,0.455416,0.463397,0.515222,0.348329,0.462664,0.493036,0.258977,0.443659,0.467741,0.464799,0.422572,0.334875,0.449949


In [18]:
model_f1_separate_params(eval_models, X_eval, best_1_3, 1, level=3, msc_col="MSC")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.223367,0.153703,0.27599,0.250378,0.126236,0.24961,0.290936,0.126748,0.275221,0.281694,0.163341,0.286626,0.274715,0.285111,0.33611,0.189576,0.290268,0.314896,0.117415,0.26786,0.290546,0.28587,0.263251,0.184415,0.281211


## TOP-2

### Parameter search

In [19]:
%%time
best_2_1 = best_model_f1_param(eval_models, X_test, list(range(5, 51)), 2, level=1)

  0%|          | 0/46 [00:00<?, ?it/s]

Bert+MP [(19, 0.7197423319361324), (16, 0.7195250178938737), (20, 0.7194064120370728), (18, 0.7193820525573368), (17, 0.7192051975087762)]
Bert+MP+class-arx [(19, 0.6569705372505149), (20, 0.656545156738123), (18, 0.6563338964339421), (22, 0.6560681913009276), (21, 0.6560569775155467)]
Bert+MP+class-zbm [(23, 0.7635670015386707), (19, 0.7634490044277964), (20, 0.7634249756719783), (18, 0.7633523628272094), (21, 0.763266955651148)]
Mathbert+CLS [(20, 0.768840612585632), (21, 0.7686639997217105), (18, 0.7684188144197607), (19, 0.7683183273976801), (22, 0.7681463048207515)]
Mathbert+MP+class-arx [(25, 0.6404400771615677), (24, 0.6403853947851195), (26, 0.6402359966381123), (29, 0.6402066801584501), (28, 0.6401586712056585)]
Mathbert+MP+class-zbm [(27, 0.7452704797579373), (25, 0.7450500683888855), (26, 0.7450390603770991), (28, 0.7448869503574335), (24, 0.7447439891724027)]
Bert+TSDAE+MP [(20, 0.803803278863893), (23, 0.8036325859679313), (22, 0.803617372281961), (19, 0.8035704703698485),

In [20]:
%%time
best_2_2 = best_model_f1_param(eval_models, X_test, list(range(5, 51)), 2, level=2)

  0%|          | 0/46 [00:00<?, ?it/s]

Bert+MP [(15, 0.5170461758388222), (16, 0.5167075631949289), (14, 0.516682265163026), (17, 0.516239280229998), (20, 0.5162065136594091)]
Bert+MP+class-arx [(18, 0.4180672792365572), (19, 0.4179972733203454), (17, 0.4178152121620171), (20, 0.4177106202031889), (16, 0.41760783512841987)]
Bert+MP+class-zbm [(19, 0.5917561976113213), (20, 0.591721548674797), (21, 0.5914094773177903), (24, 0.5912108651495177), (18, 0.5911522257520797)]
Mathbert+CLS [(27, 0.5689672223848917), (24, 0.5688761195547154), (22, 0.5688493537288103), (28, 0.5688274053238354), (23, 0.5687715527159368)]
Mathbert+MP+class-arx [(23, 0.3865866180532043), (24, 0.3865622974653941), (19, 0.3865166883424483), (20, 0.3864496172457262), (22, 0.38628302029026657)]
Mathbert+MP+class-zbm [(25, 0.5696269321419474), (24, 0.5696087348935067), (28, 0.5696027175923566), (27, 0.5692317107310692), (21, 0.5692287229134284)]
Bert+TSDAE+MP [(23, 0.6232137721367687), (25, 0.6228156027282822), (24, 0.6227183078875291), (22, 0.62265016327024

In [21]:
%%time
best_2_3 = best_model_f1_param(eval_models, X_test, list(range(5, 51)), 2, level=3)

  0%|          | 0/46 [00:00<?, ?it/s]

Bert+MP [(17, 0.3117793686683428), (14, 0.31136267916273436), (18, 0.31131719368808786), (16, 0.3111796141088749), (13, 0.31102023307494936)]
Bert+MP+class-arx [(14, 0.21757423734911727), (13, 0.2171757928201063), (16, 0.21710698542620757), (15, 0.21707103262871005), (11, 0.21658917922303828)]
Bert+MP+class-zbm [(16, 0.3853322948503197), (15, 0.38513508117181106), (17, 0.38509851845374654), (18, 0.3849594237362239), (19, 0.38488325840121385)]
Mathbert+CLS [(19, 0.35438340429843335), (18, 0.35421190029202393), (20, 0.35399670721460774), (17, 0.35388710451899424), (16, 0.3537086672444605)]
Mathbert+MP+class-arx [(13, 0.17980708182322364), (14, 0.17935011496885192), (15, 0.17929497653971832), (12, 0.17898229015121384), (16, 0.17894470290020487)]
Mathbert+MP+class-zbm [(19, 0.35729666055081755), (20, 0.3572788840148836), (22, 0.3571763455277382), (18, 0.3571463115953574), (21, 0.35681867346694957)]
Bert+TSDAE+MP [(26, 0.4143177942658418), (24, 0.41430672413468955), (25, 0.4142887846982812)

### X_test

In [22]:
model_f1_separate_params(eval_models, X_test, best_2_1, 2, level=1, msc_col="MSC")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.719742,0.656971,0.763567,0.768841,0.64044,0.74527,0.803803,0.624222,0.767697,0.813092,0.70465,0.792337,0.784082,0.795669,0.844554,0.698384,0.78294,0.828473,0.631831,0.76855,0.793845,0.793791,0.752579,0.686769,0.774333


In [23]:
model_f1_separate_params(eval_models, X_test, best_2_2, 2, level=2, msc_col="MSC")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.517046,0.418067,0.591756,0.568967,0.386587,0.569627,0.623214,0.373547,0.593555,0.62471,0.463751,0.620925,0.609577,0.618825,0.682314,0.473363,0.614236,0.654963,0.374072,0.594439,0.622945,0.620054,0.56838,0.464087,0.601483


In [24]:
model_f1_separate_params(eval_models, X_test, best_2_3, 2, level=3, msc_col="MSC")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.311779,0.217574,0.385332,0.354383,0.179807,0.357297,0.414318,0.17922,0.385498,0.40123,0.235484,0.403918,0.391121,0.400931,0.475748,0.266805,0.407148,0.444079,0.170179,0.377936,0.407798,0.405137,0.369414,0.262724,0.393047


### X_eval

In [25]:
model_f1_separate_params(eval_models, X_eval, best_2_1, 2, level=1, msc_col="MSC")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.71847,0.65432,0.762112,0.769407,0.643511,0.743407,0.805156,0.624672,0.767486,0.813215,0.704869,0.791569,0.782454,0.793007,0.844648,0.700319,0.783049,0.829281,0.63135,0.768951,0.792372,0.79223,0.750631,0.684223,0.772184


In [26]:
model_f1_separate_params(eval_models, X_eval, best_2_2, 2, level=2, msc_col="MSC")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.514299,0.418215,0.590053,0.570813,0.38799,0.567073,0.622075,0.373916,0.593995,0.623582,0.46486,0.619688,0.60814,0.617523,0.682055,0.474726,0.613642,0.65275,0.371384,0.594308,0.619705,0.617276,0.564104,0.462901,0.597277


In [27]:
model_f1_separate_params(eval_models, X_eval, best_2_3, 2, level=3, msc_col="MSC")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.311324,0.217575,0.384455,0.355008,0.181363,0.353038,0.410305,0.179485,0.385208,0.398696,0.236649,0.403892,0.389287,0.399119,0.476338,0.265671,0.406815,0.442501,0.168901,0.378549,0.408462,0.402388,0.367755,0.260034,0.392042


## TOP-3

### Parameter search

In [28]:
%%time
best_3_1 = best_model_f1_param(eval_models, X_test, list(range(5, 51)), 3, level=1)

  0%|          | 0/46 [00:00<?, ?it/s]

Bert+MP [(29, 0.7857938669478929), (26, 0.7855847644837161), (27, 0.7855461340876181), (25, 0.7855266010025154), (28, 0.7855009326248522)]
Bert+MP+class-arx [(45, 0.7305470511711816), (44, 0.7305468034909649), (46, 0.7304627980935702), (47, 0.7304096024934803), (42, 0.7303118279114618)]
Bert+MP+class-zbm [(31, 0.826569257275721), (30, 0.826553109146424), (32, 0.8263869039692386), (33, 0.8262677419778997), (37, 0.8261180367650675)]
Mathbert+CLS [(30, 0.8314159989376942), (29, 0.8312801607483838), (31, 0.8312066858507473), (32, 0.8311905752326059), (34, 0.8311793048763693)]
Mathbert+MP+class-arx [(38, 0.718252351039785), (37, 0.718082206555298), (43, 0.7180492651809384), (40, 0.7179839079893854), (44, 0.717908730570321)]
Mathbert+MP+class-zbm [(43, 0.811181045897305), (44, 0.8111766278930863), (41, 0.8109532841836836), (40, 0.8109292273443688), (47, 0.8108930991265434)]
Bert+TSDAE+MP [(31, 0.8639380200459492), (32, 0.8638786968486092), (29, 0.8637812929956791), (38, 0.8637224965847692), 

In [29]:
%%time
best_3_2 = best_model_f1_param(eval_models, X_test, list(range(5, 51)), 3, level=2)

  0%|          | 0/46 [00:00<?, ?it/s]

Bert+MP [(27, 0.5881244124916801), (26, 0.5880618450379596), (19, 0.5879942842914478), (23, 0.5879871324978523), (25, 0.5879012924685288)]
Bert+MP+class-arx [(26, 0.4896702715246832), (29, 0.48931799118600394), (25, 0.4893129744467922), (28, 0.4892951094481825), (27, 0.48913766896755806)]
Bert+MP+class-zbm [(27, 0.6627815826500538), (32, 0.6627006045759374), (31, 0.6626548767990407), (33, 0.6625799273027738), (30, 0.6625756308635753)]
Mathbert+CLS [(31, 0.6472037348225131), (40, 0.6469192568980685), (34, 0.6469164166556401), (36, 0.6468735759406716), (32, 0.6467844652265973)]
Mathbert+MP+class-arx [(25, 0.4585544141491135), (37, 0.4584218590646735), (32, 0.4583618237887047), (31, 0.4582394534592669), (26, 0.4582229470182784)]
Mathbert+MP+class-zbm [(34, 0.6430093852667652), (33, 0.642971325973405), (36, 0.6429307144804435), (35, 0.6427179371793044), (31, 0.6426446545754926)]
Bert+TSDAE+MP [(32, 0.6992101460907032), (33, 0.6991083993714635), (34, 0.6990560131524027), (31, 0.699011663400

In [30]:
%%time
best_3_3 = best_model_f1_param(eval_models, X_test, list(range(5, 51)), 3, level=3)

  0%|          | 0/46 [00:00<?, ?it/s]

Bert+MP [(22, 0.366383715634299), (25, 0.366073777357945), (23, 0.3659878246361541), (27, 0.3659395308830995), (24, 0.36590651685046666)]
Bert+MP+class-arx [(20, 0.26006276557493274), (22, 0.2595637504996078), (21, 0.25954661046004435), (23, 0.25936887646462686), (19, 0.25930761426440574)]
Bert+MP+class-zbm [(22, 0.44926327773563873), (23, 0.44860591314707454), (21, 0.4484339993610572), (26, 0.44824103322092373), (25, 0.44799780551139995)]
Mathbert+CLS [(25, 0.4164410587040901), (26, 0.41618211556595996), (23, 0.4161808261039761), (24, 0.41613549295975705), (27, 0.4160656701718101)]
Mathbert+MP+class-arx [(25, 0.21791623792399115), (26, 0.21772906155283941), (23, 0.21759800508012694), (27, 0.2174539797000877), (20, 0.21743806932348272)]
Mathbert+MP+class-zbm [(35, 0.41823620188435484), (34, 0.41817458220342113), (29, 0.4180646184158879), (37, 0.4180347401264465), (36, 0.4179390546167499)]
Bert+TSDAE+MP [(36, 0.48393213293119197), (35, 0.48379045248962393), (37, 0.4835087798288305), (27

### X_test

In [31]:
model_f1_separate_params(eval_models, X_test, best_3_1, 3, level=1, msc_col="MSC")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.785794,0.730547,0.826569,0.831416,0.718252,0.811181,0.863938,0.703397,0.831304,0.871648,0.774569,0.854629,0.847605,0.856636,0.898738,0.768673,0.843704,0.884336,0.71166,0.832133,0.854256,0.854309,0.815948,0.757455,0.836527


In [32]:
model_f1_separate_params(eval_models, X_test, best_3_2, 3, level=2, msc_col="MSC")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.588124,0.48967,0.662782,0.647204,0.458554,0.643009,0.69921,0.444675,0.667232,0.702354,0.53801,0.695214,0.684829,0.695436,0.758876,0.545648,0.686926,0.731108,0.446047,0.668957,0.696029,0.695102,0.641664,0.536523,0.674432


In [33]:
model_f1_separate_params(eval_models, X_test, best_3_3, 3, level=3, msc_col="MSC")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.366384,0.260063,0.449263,0.416441,0.217916,0.418236,0.483932,0.215983,0.448585,0.470862,0.284379,0.470858,0.457518,0.468706,0.551761,0.313393,0.472091,0.515523,0.20607,0.444064,0.475484,0.470579,0.430304,0.311503,0.457384


### X_eval

In [34]:
model_f1_separate_params(eval_models, X_eval, best_3_1, 3, level=1, msc_col="MSC")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.787794,0.72989,0.824883,0.832147,0.720621,0.809464,0.866008,0.703327,0.829452,0.871671,0.775981,0.852496,0.846072,0.854685,0.897848,0.770914,0.842464,0.885529,0.709881,0.832766,0.851821,0.852748,0.813633,0.756111,0.83511


In [35]:
model_f1_separate_params(eval_models, X_eval, best_3_2, 3, level=2, msc_col="MSC")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.586656,0.488561,0.660865,0.646472,0.459038,0.638118,0.697898,0.442173,0.6658,0.702059,0.538837,0.694241,0.68291,0.692764,0.757769,0.5461,0.684276,0.730631,0.444538,0.666755,0.692321,0.690682,0.635132,0.535434,0.67041


In [36]:
model_f1_separate_params(eval_models, X_eval, best_3_3, 3, level=3, msc_col="MSC")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.365008,0.260287,0.447117,0.416388,0.219482,0.416321,0.480256,0.214316,0.447476,0.469172,0.284067,0.470085,0.453567,0.466276,0.552584,0.311999,0.471278,0.514904,0.205657,0.44192,0.475538,0.468578,0.428242,0.309015,0.456158


## TOP-4

### Parameter search

In [37]:
%%time
best_4_1 = best_model_f1_param(eval_models, X_test, list(range(5, 51)), 4, level=1)

  0%|          | 0/46 [00:00<?, ?it/s]

Bert+MP [(34, 0.8262943645628885), (32, 0.8259578005873431), (46, 0.8259073231349172), (38, 0.8258687874353288), (40, 0.8258551403893293)]
Bert+MP+class-arx [(50, 0.7772341773095587), (47, 0.7772182579290265), (48, 0.7771691714153471), (49, 0.777130456420085), (46, 0.777037900572435)]
Bert+MP+class-zbm [(41, 0.8616602518338673), (44, 0.8616146925344607), (42, 0.8615883044484038), (40, 0.8615323008685416), (43, 0.8614660910451205)]
Mathbert+CLS [(49, 0.8679377471101749), (50, 0.8677679830238124), (48, 0.8677449901491777), (47, 0.867594674020586), (46, 0.8674020459730714)]
Mathbert+MP+class-arx [(50, 0.7671681674165352), (49, 0.7671386636402668), (47, 0.7668806890123291), (48, 0.7668765586232561), (46, 0.7668726745591695)]
Mathbert+MP+class-zbm [(50, 0.849324741540835), (48, 0.8490934209891084), (47, 0.849048757814381), (49, 0.8489908577394589), (46, 0.848616252666403)]
Bert+TSDAE+MP [(46, 0.8967931568397263), (50, 0.896769569891875), (45, 0.8965816822467804), (47, 0.8965674582259116), (

In [ ]:
%%time
best_4_2 = best_model_f1_param(eval_models, X_test, list(range(5, 51)), 4, level=2)

  0%|          | 0/46 [00:00<?, ?it/s]

In [ ]:
%%time
best_4_3 = best_model_f1_param(eval_models, X_test, list(range(5, 51)), 4, level=3)

### X_test

In [46]:
model_f1_separate_params(eval_models, X_test, best_4_1, 4, level=1, msc_col="MSC")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.826294,0.777234,0.86166,0.867938,0.767168,0.849325,0.896793,0.752555,0.866631,0.902251,0.816286,0.886556,0.882107,0.888556,0.926421,0.811274,0.877109,0.912921,0.760181,0.867897,0.88643,0.886723,0.852579,0.80162,0.871614


In [47]:
model_f1_separate_params(eval_models, X_test, best_4_2, 4, level=2, msc_col="MSC")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.636002,0.539263,0.706896,0.695431,0.510681,0.688061,0.746286,0.495588,0.711493,0.74881,0.590928,0.74156,0.731516,0.740438,0.804008,0.595009,0.730267,0.776808,0.498408,0.715118,0.74068,0.740472,0.686342,0.586223,0.719712


In [48]:
model_f1_separate_params(eval_models, X_test, best_4_3, 4, level=3, msc_col="MSC")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.406278,0.292253,0.492262,0.4602,0.247445,0.462179,0.530794,0.243766,0.493264,0.51883,0.320972,0.517056,0.505264,0.514674,0.603296,0.348522,0.516561,0.565165,0.236138,0.48904,0.521495,0.51725,0.472092,0.346748,0.501228


### X_eval

In [49]:
model_f1_separate_params(eval_models, X_eval, best_4_1, 4, level=1, msc_col="MSC")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.828222,0.776693,0.861403,0.868688,0.769517,0.847694,0.897498,0.754369,0.864792,0.90374,0.818015,0.886031,0.880893,0.887618,0.925957,0.812953,0.876162,0.913322,0.760646,0.867517,0.8845,0.885953,0.850774,0.801381,0.870827


In [50]:
model_f1_separate_params(eval_models, X_eval, best_4_2, 4, level=2, msc_col="MSC")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.633462,0.538746,0.706795,0.694367,0.512888,0.685552,0.745243,0.493826,0.710638,0.749729,0.59138,0.739354,0.728758,0.738041,0.801334,0.595248,0.728685,0.77593,0.497367,0.712058,0.737455,0.737421,0.682101,0.586466,0.71594


In [51]:
model_f1_separate_params(eval_models, X_eval, best_4_3, 4, level=3, msc_col="MSC")

,f1 (Bert+MP),f1 (Bert+MP+class-arx),f1 (Bert+MP+class-zbm),f1 (Mathbert+CLS),f1 (Mathbert+MP+class-arx),f1 (Mathbert+MP+class-zbm),f1 (Bert+TSDAE+MP),f1 (Bert+TSDAE+MP+class-arx),f1 (Bert+TSDAE+MP+class-zbm),f1 (Bert+MLM+MP),f1 (Bert+MLM+MP+class-arx),f1 (Bert+MLM+MP+class-zbm),f1 (Bert+MLM+MP+class-zbm+anch-arx),f1 (Bert+MLM+MP+class-arx+anch-arx+class-zbm),f1 (SBert),f1 (SBert+ret_class-arx),f1 (SBert+ret_class-zbm),f1 (Specter 2+MP),f1 (Specter 2.0+MP+class-arx),f1 (Specter 2.0+MP+class-zbm),f1 (Specter 2+MP+class-zbm+anch-arx),f1 (Specter 2+MP+anch-arx+class-arx+class-zbm),f1 (SGPT),f1 (SGPT+class-arx),f1 (SGPT+class-zbm)
0,0.404914,0.292245,0.489735,0.461463,0.250206,0.458868,0.528015,0.244079,0.490966,0.516125,0.321348,0.515735,0.502039,0.511802,0.602103,0.347716,0.516182,0.563661,0.235394,0.489085,0.521904,0.516647,0.469254,0.345351,0.500576
